In [2]:
## this code generates a random key from list of words, then guesses based on algortihm until it has the right word

import random

l_words=[] #define empty list of words
with open("words.txt") as file:
    for item in file:
        l_words.append(item[0:5]) #only add char 1-5, the rest are '/n' 
#create list l_words with all possible words

def get_word(l_words):  #function that picks random word to be used as key
    end =len(l_words)
    return l_words[random.randrange(end)]

def char_prob(char):  #returns probability of a given character
    prob ={'e':12.02,'t':9.1,'a':8.12,'o':7.68,'i':7.31,'n':6.95,'s':6.28
    ,'r':6.02,'h':5.92,'d':4.32,'l':3.98,'u':2.88,'c':2.71,'m':2.61,
    'f':2.3,'y':2.11,'w':2.09,'g':2.01,'p':1.82,'b':1.49,'v':1.11,
    'k':.69,'x':.17,'q':.11,'j':.1,'z':.07}
    return prob[char]

def word_prob(string): #returns sum of prob of a given word
    sum=0
    for i in string:
        sum = sum+char_prob(i)
    if duplicate(string): #this if statement is something added for optimization, ignore it 
        sum =9.67
    return sum

def duplicate(string):  #returns true if duplicates, false if no duplicates
    l=[]
    for i in string:
        if i not in l:
            l.append(i)
        else:
            return True
    return False

def evaluate(guess, key):  #function that returns 5 length string. 0 is no match, 1 is in the string, 2 is in the location
    answer = [0,0,0,0,0]
    for i in range(len(guess)): #match all letters in the key
        if guess[i] in key:
            answer[i]=1
    for i in range(len(guess)): #match all letters in the location
        if guess[i]==key[i]:
            answer[i]=2
    return answer

def next_guess_info(guesses,key): #guesses is list of guesses (list of strings) key is key string
    l_result = [] #list of evaluated strings and the string in the 5th index
    location_list=[0,0,0,0,0] #list of correct locations and chars
    not_in = []  #list of chars not in key
    let_in = []  #list of chars in key
    
    for i in guesses:   #create list of eval strings with the string on the end, eg [0,2,1,0,1,'string']
        l = evaluate(i,key)
        l.append(i)
        l_result.append(l)
    
    for i in l_result: #i is each list in l_result, for each guess
        for j in range(len(i)-1): #j is each number in each guess list j goes 0,1,2,3,4
            if i[j]==2: #if the jth element on the ith string is ==2
                location_list[j]=i[5][j]  #set the jth index of the location list to be the char at the jth index of string
            
            if i[j]==0: #if the jth element is not in the word
                not_in.append(i[5][j])
            if i[j]==1:
                let_in.append(i[5][j])

    l_return=[]
    
    l_return.append(location_list) #add location list
    l_return.append(not_in)
    l_return.append(let_in)
    return l_return  #pass as info to next_guess, is location list, not in, and in list of lists

def next_guess(info,guesses):#function that picks a next guess pass info and previous guesses  
    #SHOULD PASS L WORDS TO FUNCTION this is bad practice to use a global variable
    l_potential=list(l_words) #need to do the list() to avoid editing l_words because of pointer stuff
    
    for i in guesses: #eliminate previous guesses
        l_potential.remove(i)
    
    for i in info[1]:  #elimitate words where letters are in the not list
        for j in reversed(l_potential): #to prevent changing indices while iterating use reversed() BAD PRACTICE
            if i in j:
                l_potential.remove(j)
    
    for char in info[2]:  #remove words where yellow chars are not in the words
        for word in reversed(l_potential):
            if char not in word:
                l_potential.remove(word)
    
    for word in reversed(l_potential):   #remove words where letters are in the wrong spots
        for i in range(len(word)):
            if info[0][i] !=0 and word[i] !=info[0][i]:
                l_potential.remove(word)
                break
    
    freq=0
    #sort remaining potential guesses by their weighting
    for word in l_potential:
        if word_prob(word)>freq:
            freq=word_prob(word)
            best_guess =word
    return(best_guess)   #returns the next guess, a string
            
def guesser(initial_guess,key): #input inital guess and the key, outputs the final guess and number of guesses
    guesses =[]
    guesses.append(initial_guess)
    best_guess = initial_guess
    while best_guess != key: #while incorrect, pass to next_guess to get the next guess
        info = next_guess_info(guesses,key)
        word = next_guess(info,guesses)
        best_guess=word
        guesses.append(word)
        #print(guesses)   #uncomment this print statement if you want to see all of the guesses it made
    return best_guess,len(guesses) #return the answer and number of guesses
    
def main():  #input is nothing, returns the key and the number of guesses it took
    key = get_word(l_words) #define key as a random word
    guess = 'oaten'
    return guesser(guess,key)
    
def performance(): #a function that will tell you average number of guesses for your code, ignore it 
    count=0
    for i in range(50):
        num = main()[1]
        count+= num
    return count/50

main()

#IGNORE EVERYTHING BELOW THIS 

    
                    #33% of words in list have duplicate letters
                    #average word weight is 29.3 
                    #weight duplicates at 9.67
#performance ()  #averages 6.05 guesses to solve wordle initially
                #if words with duplicate letters get a sum=1 rather than what it actually is, average 5.07
                #
        
##below is how to get a guess if you dont know the key, practical use
# guesses = ['oaten']
# info = [['h','o','a','r',0],['e','n','t','s'],['a','o','h','r']]
# next_guess(info,guesses)

# def best_3(l_words):
#     l_pot = l_words
#     vowels = ['a','e','i','o','u']
#     for i in list(l_pot):
#         if word_prob(i)<30:
#             l_pot.remove(i)
    
#     for i in list(l_pot):
#         vowel_count =0
#         for j in vowels:
#             if j in i:
#                 vowel_count +=1
#         if vowel_count !=2:
#             l_pot.remove(i)  #remove all words with not equal to 2 vowels
#     l=[]
#     count=0
#     count_n=0
#     for i in list(l_pot):
#         for j in list(l_pot):
#             for k in list(l_pot):
#                 l.append(i)
#                 l.append(j)
#                 l.append(k)
#                 for var in l:
#                     count_n = count_n +word_prob(var)
#                 if count_n>count:
#                     combo=l
    
#     return combo

# best_3(l_words)




['oaten', 'osier']
['oaten', 'osier', 'hosed']
['oaten', 'osier', 'hosed', 'louse']
['oaten', 'osier', 'hosed', 'louse', 'socle']
['oaten', 'osier', 'hosed', 'louse', 'socle', 'solve']


('solve', 6)